# New approach to decision trees for transition

In [1]:
using CovidSim

┌ Info: Precompiling CovidSim [top-level]
└ @ Base loading.jl:1278


In [2]:
using StatsBase
using DelimitedFiles
using Distributions

In [3]:
ilmat = readdlm("../data/ilmtestdata.csv", ',',Int, header=true)[1]
ilmat = repeat(ilmat, 10_000)
refresh = copy(ilmat)

160000×4 Array{Int64,2}:
 1  2  0   0
 1  3  0   0
 1  4  0   0
 1  5  0   0
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 ⋮        
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 3  5  6  15
 4  3  8  15
 4  4  8  19

## New Approach to Decision Trees

In [4]:
dectreefilename="../parameters/dec_tree_all_25.csv"

"../parameters/dec_tree_all_25.csv"

In [5]:
using JSON
using Pkg.TOML
using YAML

In [6]:
dt, all_decpoints = CovidSim.setup_dt_old(dectreefilename)

(Any[(dec_points = Dict(9 => [(2, 1), (2, 2), (2, 3)],14 => [(3, 2), (3, 3), (3, 4)],25 => [(5, 3), (5, 4)],19 => [(4, 4)],5 => [(1, 1)]), tree = Dict{Tuple{Int64,Int64},Array{Any,1}}((2, 3) => [CovidSim.Branch(7, 7, 0.95, (3, 3), "sick", "sick"), CovidSim.Branch(7, 8, 0.05, (3, 4), "sick", "severe")],(3, 2) => [CovidSim.Branch(6, 3, 1.0, (0, 0), "mild", "recovered")],(3, 3) => [CovidSim.Branch(7, 3, 0.85, (0, 0), "sick", "recovered"), CovidSim.Branch(7, 7, 0.12, (5, 3), "sick", "sick"), CovidSim.Branch(7, 8, 0.03, (4, 4), "sick", "severe")],(2, 2) => [CovidSim.Branch(6, 6, 1.0, (3, 2), "mild", "mild")],(1, 1) => [CovidSim.Branch(5, 5, 0.4, (2, 1), "nil", "nil"), CovidSim.Branch(5, 6, 0.5, (2, 2), "nil", "mild"), CovidSim.Branch(5, 7, 0.1, (2, 3), "nil", "sick")],(4, 4) => [CovidSim.Branch(8, 3, 0.891, (0, 0), "severe", "recovered"), CovidSim.Branch(8, 8, 0.106, (5, 4), "severe", "severe"), CovidSim.Branch(8, 4, 0.003, (0, 5), "severe", "dead")],(5, 3) => [CovidSim.Branch(7, 3, 0.976, 

In [7]:
newdict1 = Dict(string(k) => dt[1].tree[k] for k in keys(dt[1].tree))

Dict{String,Array{Any,1}} with 10 entries:
  "(5, 4)" => Any[Branch(8, 3, 0.91, (0, 0), "severe", "recovered"), Branch(8, …
  "(3, 2)" => Any[Branch(6, 3, 1.0, (0, 0), "mild", "recovered")]
  "(3, 3)" => Any[Branch(7, 3, 0.85, (0, 0), "sick", "recovered"), Branch(7, 7,…
  "(2, 2)" => Any[Branch(6, 6, 1.0, (3, 2), "mild", "mild")]
  "(1, 1)" => Any[Branch(5, 5, 0.4, (2, 1), "nil", "nil"), Branch(5, 6, 0.5, (2…
  "(4, 4)" => Any[Branch(8, 3, 0.891, (0, 0), "severe", "recovered"), Branch(8,…
  "(3, 4)" => Any[Branch(8, 3, 0.692, (0, 0), "severe", "recovered"), Branch(8,…
  "(5, 3)" => Any[Branch(7, 3, 0.976, (0, 0), "sick", "recovered"), Branch(7, 4…
  "(2, 1)" => Any[Branch(5, 3, 0.9, (0, 0), "nil", "recovered"), Branch(5, 7, 0…
  "(2, 3)" => Any[Branch(7, 7, 0.95, (3, 3), "sick", "sick"), Branch(7, 8, 0.05…

In [8]:
keys(dt[1].tree)

Base.KeySet for a Dict{Tuple{Int64,Int64},Array{Any,1}} with 10 entries. Keys:
  (2, 3)
  (3, 2)
  (3, 3)
  (2, 2)
  (1, 1)
  (4, 4)
  (5, 3)
  (5, 4)
  (2, 1)
  (3, 4)

In [9]:
JSON.json(dt[1].tree)

"{\"(2, 3)\":[{\"fromcond\":7,\"tocond\":7,\"pr\":0.95,\"next\":[3,3],\"fromcondname\":\"sick\",\"tocondname\":\"sick\"},{\"fromcond\":7,\"tocond\":8,\"pr\":0.05,\"next\":[3,4],\"fromcondname\":\"sick\",\"tocondname\":\"severe\"}],\"(3, 2)\":[{\"fromcond\":6,\"tocond\":3,\"pr\":1.0,\"next\":[0,0],\"fromcondname\":\"mild\",\"tocondname\":\"recovered\"}],\"(3, 3)\":[{\"fromcond\":7,\"tocond\":3,\"pr\":0.85,\"next\":[0,0],\"fromcondname\":\"sick\",\"tocondname\":\"recovered\"},{\"fromcond\":7,\"tocond\":7,\"pr\":0.12,\"next\":[5,3],\"fromcondname\":\"sick\",\"tocondname\":\"sick\"},{\"fromcond\":7,\"tocond\":8,\"pr\":0.03,\"next\":[4,4],\"fromcondname\":\"sick\",\"tocondname\":\"severe\"}],\"(2, 2)\":[{\"fromcond\":6,\"tocond\":6,\"pr\":1.0,\"next\":[3,2],\"fromcondname\":\"mild\",\"tocondname\":\"mild\"}],\"(1, 1)\":[{\"fromcond\":5,\"tocond\":5,\"pr\":0.4,\"next\":[2,1],\"fromcondname\":\"nil\",\"tocondname\":\"nil\"},{\"fromcond\":5,\"tocond\":6,\"pr\":0.5,\"next\":[2,2],\"fromcondname

In [10]:
JSON.json(dt)

"[{\"dec_points\":{\"9\":[[2,1],[2,2],[2,3]],\"14\":[[3,2],[3,3],[3,4]],\"25\":[[5,3],[5,4]],\"19\":[[4,4]],\"5\":[[1,1]]},\"tree\":{\"(2, 3)\":[{\"fromcond\":7,\"tocond\":7,\"pr\":0.95,\"next\":[3,3],\"fromcondname\":\"sick\",\"tocondname\":\"sick\"},{\"fromcond\":7,\"tocond\":8,\"pr\":0.05,\"next\":[3,4],\"fromcondname\":\"sick\",\"tocondname\":\"severe\"}],\"(3, 2)\":[{\"fromcond\":6,\"tocond\":3,\"pr\":1.0,\"next\":[0,0],\"fromcondname\":\"mild\",\"tocondname\":\"recovered\"}],\"(3, 3)\":[{\"fromcond\":7,\"tocond\":3,\"pr\":0.85,\"next\":[0,0],\"fromcondname\":\"sick\",\"tocondname\":\"recovered\"},{\"fromcond\":7,\"tocond\":7,\"pr\":0.12,\"next\":[5,3],\"fromcondname\":\"sick\",\"tocondname\":\"sick\"},{\"fromcond\":7,\"tocond\":8,\"pr\":0.03,\"next\":[4,4],\"fromcondname\":\"sick\",\"tocondname\":\"severe\"}],\"(2, 2)\":[{\"fromcond\":6,\"tocond\":6,\"pr\":1.0,\"next\":[3,2],\"fromcondname\":\"mild\",\"tocondname\":\"mild\"}],\"(1, 1)\":[{\"fromcond\":5,\"tocond\":5,\"pr\":0.4,\"

In [11]:
TOML.print(newdict1)

"(5, 4)" = ["CovidSim.Branch(8, 3, 0.91, (0, 0), \"severe\", \"recovered\")", "CovidSim.Branch(8, 4, 0.09, (0, 5), \"severe\", \"dead\")"]
"(3, 2)" = ["CovidSim.Branch(6, 3, 1.0, (0, 0), \"mild\", \"recovered\")"]
"(3, 3)" = ["CovidSim.Branch(7, 3, 0.85, (0, 0), \"sick\", \"recovered\")", "CovidSim.Branch(7, 7, 0.12, (5, 3), \"sick\", \"sick\")", "CovidSim.Branch(7, 8, 0.03, (4, 4), \"sick\", \"severe\")"]
"(2, 2)" = ["CovidSim.Branch(6, 6, 1.0, (3, 2), \"mild\", \"mild\")"]
"(1, 1)" = ["CovidSim.Branch(5, 5, 0.4, (2, 1), \"nil\", \"nil\")", "CovidSim.Branch(5, 6, 0.5, (2, 2), \"nil\", \"mild\")", "CovidSim.Branch(5, 7, 0.1, (2, 3), \"nil\", \"sick\")"]
"(4, 4)" = ["CovidSim.Branch(8, 3, 0.891, (0, 0), \"severe\", \"recovered\")", "CovidSim.Branch(8, 8, 0.106, (5, 4), \"severe\", \"severe\")", "CovidSim.Branch(8, 4, 0.003, (0, 5), \"severe\", \"dead\")"]
"(3, 4)" = ["CovidSim.Branch(8, 3, 0.692, (0, 0), \"severe\", \"recovered\")", "CovidSim.Branch(8, 8, 0.302, (4, 4), \"severe\", \"se

In [12]:
# this is json from the YAML
jsontxt = """
[
  {
    "nodes": [
      {
        "branches": [
          {
            "pr": 1.0, 
            "tocond": 7, 
            "id": "to 7", 
            "next": "(0, 0)"
          }, 
          {
            "pr": 0.85, 
            "tocond": 7, 
            "id": "to 7", 
            "next": "(0, 0)"
          }, 
          {
            "pr": 0.15, 
            "tocond": 8, 
            "id": "to 8", 
            "next": "(4, 4)"
          }
        ], 
        "id": "(14, 6)"
      }
    ], 
    "agegrp": 1
  }, 
  {
    "nodes": [
      {
        "branches": [
          {
            "pr": 1.0, 
            "tocond": 7, 
            "id": "to 7", 
            "next": "(0, 0)"
          }
        ], 
        "id": "(14, 6)"
      }, 
      {
        "branches": [
          {
            "pr": 0.85, 
            "tocond": 7, 
            "id": "to 7", 
            "next": "(0, 0)"
          }, 
          {
            "pr": 0.15, 
            "tocond": 8, 
            "id": "to 8", 
            "next": "(4, 4)"
          }
        ], 
        "id": "(14, 7)"
      }
    ], 
    "agegrp": 2
  }
]
"""

"[\n  {\n    \"nodes\": [\n      {\n        \"branches\": [\n          {\n            \"pr\": 1.0, \n            \"tocond\": 7, \n            \"id\": \"to 7\", \n            \"next\": \"(0, 0)\"\n          }, \n          {\n            \"pr\": 0.85, \n            \"tocond\": 7, \n            \"id\": \"to 7\", \n            \"next\": \"(0, 0)\"\n          }, \n          {\n            \"pr\": 0.15, \n            \"tocond\": 8, \n            \"id\": \"to 8\", \n            \"next\": \"(4, 4)\"\n          }\n        ], \n        \"id\": \"(14, 6)\"\n      }\n    ], \n    \"agegrp\": 1\n  }, \n  {\n    \"nodes\": [\n      {\n        \"branches\": [\n          {\n            \"pr\": 1.0, \n            \"tocond\": 7, \n            \"id\": \"to 7\", \n            \"next\": \"(0, 0)\"\n          }\n        ], \n        \"id\": \"(14, 6)\"\n      }, \n      {\n        \"branches\": [\n          {\n            \"pr\": 0.85, \n            \"tocond\": 7, \n            \"id\": \"to 7\", \n         

In [13]:
tst = JSON.parse(jsontxt)

2-element Array{Any,1}:
 Dict{String,Any}("nodes" => Any[Dict{String,Any}("branches" => Any[Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 1.0), Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 0.85), Dict{String,Any}("tocond" => 8,"next" => "(4, 4)","id" => "to 8","pr" => 0.15)],"id" => "(14, 6)")],"agegrp" => 1)
 Dict{String,Any}("nodes" => Any[Dict{String,Any}("branches" => Any[Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 1.0)],"id" => "(14, 6)"), Dict{String,Any}("branches" => Any[Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 0.85), Dict{String,Any}("tocond" => 8,"next" => "(4, 4)","id" => "to 8","pr" => 0.15)],"id" => "(14, 7)")],"agegrp" => 2)

In [14]:
keys(tst[2])

Base.KeySet for a Dict{String,Any} with 2 entries. Keys:
  "nodes"
  "agegrp"

In [15]:
tst[2]["nodes"]

2-element Array{Any,1}:
 Dict{String,Any}("branches" => Any[Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 1.0)],"id" => "(14, 6)")
 Dict{String,Any}("branches" => Any[Dict{String,Any}("tocond" => 7,"next" => "(0, 0)","id" => "to 7","pr" => 0.85), Dict{String,Any}("tocond" => 8,"next" => "(4, 4)","id" => "to 8","pr" => 0.15)],"id" => "(14, 7)")

In [16]:
tst[2]["agegrp"]

2

In [17]:
# this is hand authored json
jsonhand = """
{
"1":{
      "(5,5)":[{"tocond":5,"pr":0.2,"next":[9,5]},
                {"tocond":6,"pr":0.65,"next":[9,6]},
                {"tocond":7,"pr":0.15,"next":[9,7]}],
      "(9,5)":[{"tocond":3,"pr":0.8,"next":[0,0]},
                {"tocond":7,"pr":0.2,"next":[14,7]}],
      "(9,6)":[{"tocond":6,"pr":1.0,"next":[14,6]}],
      "(9,7)":[{"tocond":7,"pr":0.85,"next":[14,7]},
                {"tocond":8,"pr":0.15,"next":[14,8]}],
      "(14,6)":[{"tocond":3,"pr":1.0,"next":[0,0]}],
      "(14,7)":[{"tocond":3,"pr":0.85,"next":[0,0]},
                {"tocond":8,"pr":0.15,"next":[19,8]}],
      "(14,8)":[{"tocond":3,"pr":0.45,"next":[0,0]},
                {"tocond":8,"pr":0.5,"next":[19,8]},
                {"tocond":4,"pr":0.05,"next":[0,5]}],
      "(19,8)":[{"tocond":3,"pr":0.9,"next":[0,0]},
                {"tocond":4,"pr":0.1,"next":[0,5]}]
    },
 "2":{
      "(5,5)":[{"tocond":5,"pr":0.2,"next":[9,5]},
                {"tocond":6,"pr":0.65,"next":[9,6]},
                {"tocond":7,"pr":0.15,"next":[9,7]}],
      "(9,5)":[{"tocond":3,"pr":0.8,"next":[0,0]},
                {"tocond":7,"pr":0.2,"next":[14,7]}],
      "(9,6)":[{"tocond":6,"pr":1.0,"next":[14,6]}],
      "(9,7)":[{"tocond":7,"pr":0.85,"next":[14,7]},
                {"tocond":8,"pr":0.15,"next":[14,8]}],
      "(14,6)":[{"tocond":3,"pr":1.0,"next":[0,0]}],
      "(14,7)":[{"tocond":3,"pr":0.8,"next":[10,0]},
                {"tocond":8,"pr":0.2,"next":[19,8]}],
      "(14,8)":[{"tocond":3,"pr":0.35,"next":[0,0]},
                {"tocond":8,"pr":0.55,"next":[19,8]},
                {"tocond":4,"pr":0.1,"next":[0,5]}],
      "(19,8)":[{"tocond":3,"pr":0.85,"next":[0,0]},
                {"tocond":4,"pr":0.15,"next":[0,5]}]
      },
 "3":{
      "(5,5)":[{"tocond":5,"pr":0.2,"next":[9,5]},
                {"tocond":6,"pr":0.6,"next":[9,6]},
                {"tocond":7,"pr":0.2,"next":[9,7]}],
      "(9,5)":[{"tocond":3,"pr":0.7,"next":[0,0]},
                {"tocond":7,"pr":0.3,"next":[14,7]}],
      "(9,6)":[{"tocond":6,"pr":1.0,"next":[14,6]}],
      "(9,7)":[{"tocond":7,"pr":0.85,"next":[14,7]},
                {"tocond":8,"pr":0.15,"next":[14,8]}],
      "(14,6)":[{"tocond":3,"pr":1.0,"next":[0,0]}],
      "(14,7)":[{"tocond":3,"pr":0.8,"next":[10,0]},
                {"tocond":8,"pr":0.2,"next":[19,8]}],
      "(14,8)":[{"tocond":3,"pr":0.35,"next":[0,0]},
                {"tocond":8,"pr":0.55,"next":[19,8]},
                {"tocond":4,"pr":0.1,"next":[0,5]}],
      "(19,8)":[{"tocond":3,"pr":0.85,"next":[0,0]},
                {"tocond":4,"pr":0.15,"next":[0,5]}]
      },
 "4":{
      "(5,5)":[{"tocond":5,"pr":0.2,"next":[9,5]},
                {"tocond":6,"pr":0.65,"next":[9,6]},
                {"tocond":7,"pr":0.15,"next":[9,7]}],
      "(9,5)":[{"tocond":3,"pr":0.8,"next":[0,0]},
                {"tocond":7,"pr":0.2,"next":[14,7]}],
      "(9,6)":[{"tocond":6,"pr":1.0,"next":[14,6]}],
      "(9,7)":[{"tocond":7,"pr":0.85,"next":[14,7]},
                {"tocond":8,"pr":0.15,"next":[14,8]}],
      "(14,6)":[{"tocond":3,"pr":1.0,"next":[0,0]}],
      "(14,7)":[{"tocond":3,"pr":0.8,"next":[10,0]},
                {"tocond":8,"pr":0.2,"next":[19,8]}],
      "(14,8)":[{"tocond":3,"pr":0.35,"next":[0,0]},
                {"tocond":8,"pr":0.55,"next":[19,8]},
                {"tocond":4,"pr":0.1,"next":[0,5]}],
      "(19,8)":[{"tocond":3,"pr":0.85,"next":[0,0]},
                {"tocond":4,"pr":0.15,"next":[0,5]}]
      },
 "5":{
      "(5,5)":[{"tocond":5,"pr":0.2,"next":[9,5]},
                {"tocond":6,"pr":0.65,"next":[9,6]},
                {"tocond":7,"pr":0.15,"next":[9,7]}],
      "(9,5)":[{"tocond":3,"pr":0.8,"next":[0,0]},
                {"tocond":7,"pr":0.2,"next":[14,7]}],
      "(9,6)":[{"tocond":6,"pr":1.0,"next":[14,6]}],
      "(9,7)":[{"tocond":7,"pr":0.85,"next":[14,7]},
                {"tocond":8,"pr":0.15,"next":[14,8]}],
      "(14,6)":[{"tocond":3,"pr":1.0,"next":[0,0]}],
      "(14,7)":[{"tocond":3,"pr":0.8,"next":[10,0]},
                {"tocond":8,"pr":0.2,"next":[19,8]}],
      "(14,8)":[{"tocond":3,"pr":0.35,"next":[0,0]},
                {"tocond":8,"pr":0.55,"next":[19,8]},
                {"tocond":4,"pr":0.1,"next":[0,5]}],
      "(19,8)":[{"tocond":3,"pr":0.85,"next":[0,0]},
                {"tocond":4,"pr":0.15,"next":[0,5]}]
      }

}
"""

"{\n\"1\":{\n      \"(5,5)\":[{\"tocond\":5,\"pr\":0.2,\"next\":[9,5]},\n                {\"tocond\":6,\"pr\":0.65,\"next\":[9,6]},\n                {\"tocond\":7,\"pr\":0.15,\"next\":[9,7]}],\n      \"(9,5)\":[{\"tocond\":3,\"pr\":0.8,\"next\":[0,0]},\n                {\"tocond\":7,\"pr\":0.2,\"next\":[14,7]}],\n      \"(9,6)\":[{\"tocond\":6,\"pr\":1.0,\"next\":[14,6]}],\n      \"(9,7)\":[{\"tocond\":7,\"pr\":0.85,\"next\":[14,7]},\n                {\"tocond\":8,\"pr\":0.15,\"next\":[14,8]}],\n      \"(14,6)\":[{\"tocond\":3,\"pr\":1.0,\"next\":[0,0]}],\n      \"(14,7)\":[{\"tocond\":3,\"pr\":0.85,\"next\":[0,0]},\n                {\"tocond\":8,\"pr\":0.15,\"next\":[19,8]}],\n      \"(14,8)\":[{\"tocond\":3,\"pr\":0.45,\"next\":[0,0]},\n                {\"tocond\":8,\"pr\":0.5,\"next\":[19,8]},\n                {\"tocond\":4,\"pr\":0.05,\"next\":[0,5]}],\n      \"(19,8)\":[{\"tocond\":3,\"pr\":0.9,\"next\":[0,0]},\n                {\"tocond\":4,\"pr\":0.1,\"next\":[0,5]}]\n    },\n \

In [18]:
hand = JSON.parse(jsonhand)

Dict{String,Any} with 5 entries:
  "4" => Dict{String,Any}("(9,6)"=>Any[Dict{String,Any}("tocond"=>6,"next"=>Any…
  "1" => Dict{String,Any}("(9,6)"=>Any[Dict{String,Any}("tocond"=>6,"next"=>Any…
  "5" => Dict{String,Any}("(9,6)"=>Any[Dict{String,Any}("tocond"=>6,"next"=>Any…
  "2" => Dict{String,Any}("(9,6)"=>Any[Dict{String,Any}("tocond"=>6,"next"=>Any…
  "3" => Dict{String,Any}("(9,6)"=>Any[Dict{String,Any}("tocond"=>6,"next"=>Any…

In [19]:
hand["1"]

Dict{String,Any} with 8 entries:
  "(9,6)"  => Any[Dict{String,Any}("tocond"=>6,"next"=>Any[14, 6],"pr"=>1.0)]
  "(9,5)"  => Any[Dict{String,Any}("tocond"=>3,"next"=>Any[0, 0],"pr"=>0.8), Di…
  "(14,6)" => Any[Dict{String,Any}("tocond"=>3,"next"=>Any[0, 0],"pr"=>1.0)]
  "(9,7)"  => Any[Dict{String,Any}("tocond"=>7,"next"=>Any[14, 7],"pr"=>0.85), …
  "(5,5)"  => Any[Dict{String,Any}("tocond"=>5,"next"=>Any[9, 5],"pr"=>0.2), Di…
  "(14,8)" => Any[Dict{String,Any}("tocond"=>3,"next"=>Any[0, 0],"pr"=>0.45), D…
  "(14,7)" => Any[Dict{String,Any}("tocond"=>3,"next"=>Any[0, 0],"pr"=>0.85), D…
  "(19,8)" => Any[Dict{String,Any}("tocond"=>3,"next"=>Any[0, 0],"pr"=>0.9), Di…

In [20]:
hand["1"]["(5,5)"]

3-element Array{Any,1}:
 Dict{String,Any}("tocond" => 5,"next" => Any[9, 5],"pr" => 0.2)
 Dict{String,Any}("tocond" => 6,"next" => Any[9, 6],"pr" => 0.65)
 Dict{String,Any}("tocond" => 7,"next" => Any[9, 7],"pr" => 0.15)

In [21]:
using YAML

In [22]:
YAML.write_file("../parameters/test-output.yml", hand)

In [23]:
newyaml = """
4:
  (9,6):
    - tocond: 6
      next:
        - 14
        - 6
      pr: 1.0
  (9,5):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.8
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.2
  (14,6):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 1.0
  (9,7):
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.85
    - tocond: 8
      next:
        - 14
        - 8
      pr: 0.15
  (5,5):
    - tocond: 5
      next:
        - 9
        - 5
      pr: 0.2
    - tocond: 6
      next:
        - 9
        - 6
      pr: 0.65
    - tocond: 7
      next:
        - 9
        - 7
      pr: 0.15
  (14,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.35
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.55
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.1
  (14,7):
    - tocond: 3
      next:
        - 10
        - 0
      pr: 0.8
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.2
  (19,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.85
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.15
1:
  (9,6):
    - tocond: 6
      next:
        - 14
        - 6
      pr: 1.0
  (9,5):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.8
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.2
  (14,6):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 1.0
  (9,7):
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.85
    - tocond: 8
      next:
        - 14
        - 8
      pr: 0.15
  (5,5):
    - tocond: 5
      next:
        - 9
        - 5
      pr: 0.2
    - tocond: 6
      next:
        - 9
        - 6
      pr: 0.65
    - tocond: 7
      next:
        - 9
        - 7
      pr: 0.15
  (14,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.45
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.5
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.05
  (14,7):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.85
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.15
  (19,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.9
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.1
5:
  (9,6):
    - tocond: 6
      next:
        - 14
        - 6
      pr: 1.0
  (9,5):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.8
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.2
  (14,6):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 1.0
  (9,7):
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.85
    - tocond: 8
      next:
        - 14
        - 8
      pr: 0.15
  (5,5):
    - tocond: 5
      next:
        - 9
        - 5
      pr: 0.2
    - tocond: 6
      next:
        - 9
        - 6
      pr: 0.65
    - tocond: 7
      next:
        - 9
        - 7
      pr: 0.15
  (14,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.35
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.55
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.1
  (14,7):
    - tocond: 3
      next:
        - 10
        - 0
      pr: 0.8
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.2
  (19,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.85
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.15
2:
  (9,6):
    - tocond: 6
      next:
        - 14
        - 6
      pr: 1.0
  (9,5):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.8
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.2
  (14,6):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 1.0
  (9,7):
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.85
    - tocond: 8
      next:
        - 14
        - 8
      pr: 0.15
  (5,5):
    - tocond: 5
      next:
        - 9
        - 5
      pr: 0.2
    - tocond: 6
      next:
        - 9
        - 6
      pr: 0.65
    - tocond: 7
      next:
        - 9
        - 7
      pr: 0.15
  (14,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.35
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.55
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.1
  (14,7):
    - tocond: 3
      next:
        - 10
        - 0
      pr: 0.8
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.2
  (19,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.85
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.15
3:
  (9,6):
    - tocond: 6
      next:
        - 14
        - 6
      pr: 1.0
  (9,5):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.7
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.3
  (14,6):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 1.0
  (9,7):
    - tocond: 7
      next:
        - 14
        - 7
      pr: 0.85
    - tocond: 8
      next:
        - 14
        - 8
      pr: 0.15
  (5,5):
    - tocond: 5
      next:
        - 9
        - 5
      pr: 0.2
    - tocond: 6
      next:
        - 9
        - 6
      pr: 0.6
    - tocond: 7
      next:
        - 9
        - 7
      pr: 0.2
  (14,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.35
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.55
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.1
  (14,7):
    - tocond: 3
      next:
        - 10
        - 0
      pr: 0.8
    - tocond: 8
      next:
        - 19
        - 8
      pr: 0.2
  (19,8):
    - tocond: 3
      next:
        - 0
        - 0
      pr: 0.85
    - tocond: 4
      next:
        - 0
        - 5
      pr: 0.15
"""

"4:\n  (9,6):\n    - tocond: 6\n      next:\n        - 14\n        - 6\n      pr: 1.0\n  (9,5):\n    - tocond: 3\n      next:\n        - 0\n        - 0\n      pr: 0.8\n    - tocond: 7\n      next:\n        - 14\n        - 7\n      pr: 0.2\n  (14,6):\n    - tocond: 3\n      next:\n        - 0\n        - 0\n      pr: 1.0\n  (9,7):\n    - tocond: 7\n      next:\n        - 14\n        - 7\n      pr: 0.85\n    - tocond: 8\n      next:\n        - 14\n        - 8\n      pr: 0.15\n  (5,5):\n    - tocond: 5\n      next:\n        - 9\n        - 5\n      pr: 0.2\n    - tocond: 6\n      next:\n        - 9\n        - 6\n      pr: 0.65\n    - tocond: 7\n      next:\n        - 9\n        - 7\n      pr: 0.15\n  (14,8):\n    - tocond: 3\n      next:\n        - 0\n        - 0\n      pr: 0.35\n    - tocond: 8\n      next:\n        - 19\n        - 8\n      pr: 0.55\n    - tocond: 4\n      next:\n        - 0\n        - 5\n      pr: 0.1\n  (14,7):\n    - tocond: 3\n      next:\n        - 10\n        - 0\n  

In [24]:
roundtrip_yaml = YAML.load(newyaml)

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  2 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  3 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  5 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  1 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…

In [25]:
roundtrip_yaml[1]

Dict{Any,Any} with 8 entries:
  "(9,6)"  => Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"pr"=>1.0)]
  "(9,5)"  => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.8), Dict("t…
  "(14,6)" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>1.0)]
  "(9,7)"  => Dict{Any,Any}[Dict("tocond"=>7,"next"=>[14, 7],"pr"=>0.85), Dict(…
  "(5,5)"  => Dict{Any,Any}[Dict("tocond"=>5,"next"=>[9, 5],"pr"=>0.2), Dict("t…
  "(14,8)" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.45), Dict("…
  "(14,7)" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.85), Dict("…
  "(19,8)" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.9), Dict("t…

In [26]:
roundtrip_yaml[1]["(9,7)"]

2-element Array{Dict{Any,Any},1}:
 Dict("tocond" => 7,"next" => [14, 7],"pr" => 0.85)
 Dict("tocond" => 8,"next" => [14, 8],"pr" => 0.15)

In [28]:
dense_yaml = """
4:
  (9,6):
    - {tocond: 6, next: [14,6], pr: 1.0}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.8}
    - {tocond: 7, next: [14,7], pr: 0.2}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.85}
    - {tocond: 8, next: [14, 8], pr: 0.15}
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.65}
    - {tocond: 7, next: [9, 7], pr: 0.15}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.35}
    - {tocond: 8, next: [19, 8], pr: 0.55}
    - {tocond: 4, next: [0,5], pr: 0.1}
  (14,7):
    - {tocond: 3, next: [0, 0], pr: 0.8}
    - {tocond: 8, next: [19, 8], pr: 0.2}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 4, next: [0,5], pr: 0.15}
1:
  (9,6):
    - {tocond: 6, next: [14, 6], pr: 1.0}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.8}
    - {tocond: 7, next: [14, 7], pr: 0.2}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.85}
    - {tocond: 8, next: [14, 8], pr: 0.15}
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.65}
    - {tocond: 7, next: [9, 7], pr: 0.15}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.45}
    - {tocond: 8, next: [19, 8], pr: 0.5}
    - {tocond: 4, next: [0,5], pr: 0.05}
  (14,7):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 8, next: [19, 8], pr: 0.15}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.9}
    - {tocond: 4, next: [0,5], pr: 0.1}
5:
  (9,6):
    - {tocond: 6, next: [14,6], pr: 1.0}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.8}
    - {tocond: 7, next: [14,7], pr: 0.2}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.85}
    - {tocond: 8, next: [14, 8], pr: 0.15}
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.65}
    - {tocond: 7, next: [9, 7], pr: 0.15}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.35}
    - {tocond: 8, next: [19, 8], pr: 0.55}
    - {tocond: 4, next: [0,5], pr: 0.1}
  (14,7):
    - {tocond: 3, next: [0, 0], pr: 0.8}
    - {tocond: 8, next: [19, 8], pr: 0.2}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 4, next: [0,5], pr: 0.15}
2:
  (9,6):
    - {tocond: 6, next: [14,6], pr: 1.0}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.8}
    - {tocond: 7, next: [14, 7], pr: 0.2}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.85}
    - {tocond: 8, next: [14, 7], pr: 0.15}
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.65}
    - {tocond: 7, next: [9, 7], pr: 0.15}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.35}
    - {tocond: 8, next: [19, 8], pr: 0.55}
    - {tocond: 4, next: [0,5], pr: 0.1}
  (14,7):
    - {tocond: 3, next: [0, 0], pr: 0.8}
    - {tocond: 8, next: [19, 8], pr: 0.2}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 4, next: [0,5], pr: 0.15}
3:
  (9,6):
    - {tocond: 6, next: [14,6], pr: 1.0}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.8}
    - {tocond: 7, next: [14,7], pr: 0.2}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.85}
    - {tocond: 8, next: [14, 8], pr: 0.15}
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.65}
    - {tocond: 7, next: [9, 7], pr: 0.15}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.35}
    - {tocond: 8, next: [19, 8], pr: 0.55}
    - {tocond: 4, next: [0,5], pr: 0.1}
  (14,7):
    - {tocond: 3, next: [0, 0], pr: 0.8}
    - {tocond: 8, next: [19, 8], pr: 0.2}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 4, next: [0,5], pr: 0.15}
"""

"4:\n  (9,6):\n    - {tocond: 6, next: [14,6], pr: 1.0}\n  (9,5):\n    - {tocond: 3, next: [0,0], pr: 0.8}\n    - {tocond: 7, next: [14,7], pr: 0.2}\n  (14,6):\n    - {tocond: 3, next: [0,0], pr: 1.0}\n  (9,7):\n    - {tocond: 7, next: [14, 7], pr: 0.85}\n    - {tocond: 8, next: [14, 8], pr: 0.15}\n  (5,5):\n    - {tocond: 5, next: [9, 5], pr: 0.2}\n    - {tocond: 6, next: [9, 6], pr: 0.65}\n    - {tocond: 7, next: [9, 7], pr: 0.15}\n  (14,8):\n    - {tocond: 3, next: [0,0], pr: 0.35}\n    - {tocond: 8, next: [19, 8], pr: 0.55}\n    - {tocond: 4, next: [0,5], pr: 0.1}\n  (14,7):\n    - {tocond: 3, next: [0, 0], pr: 0.8}\n    - {tocond: 8, next: [19, 8], pr: 0.2}\n  (19,8):\n    - {tocond: 3, next: [0,0], pr: 0.85}\n    - {tocond: 4, next: [0,5], pr: 0.15}\n1:\n  (9,6):\n    - {tocond: 6, next: [14, 6], pr: 1.0}\n  (9,5):\n    - {tocond: 3, next: [0,0], pr: 0.8}\n    - {tocond: 7, next: [14, 7], pr: 0.2}\n  (14,6):\n    - {tocond: 3, next: [0,0], pr: 1.0}\n  (9,7):\n    - {tocond: 7, ne

In [29]:
dense_literal_yaml = YAML.load(dense_yaml)

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  2 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  3 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  5 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…
  1 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…

In [46]:
hand_yaml = """
3:
  (5,5):
    - {tocond: 5, next: [9, 5], pr: 0.2}
    - {tocond: 6, next: [9, 6], pr: 0.7}
    - {tocond: 7, next: [9, 7], pr: 0.1}
  (9,5):
    - {tocond: 3, next: [0,0], pr: 0.85}
    - {tocond: 7, next: [14, 7], pr: 0.15}
  (9,6):
    - {tocond: 6, next: [14,6], pr: 1.0}
  (9,7):
    - {tocond: 7, next: [14, 7], pr: 0.9}
    - {tocond: 8, next: [14, 8], pr: 0.1}
  (14,6):
    - {tocond: 3, next: [0,0], pr: 1.0}
  (14,7):
    - {tocond: 3, next: [0, 0], pr: 0.83}
    - {tocond: 8, next: [19, 7], pr: 0.1}
    - {tocond: 8, next: [19, 8], pr: 0.07}
  (14,8):
    - {tocond: 3, next: [0,0], pr: 0.474}
    - {tocond: 8, next: [19, 8], pr: 0.514}
    - {tocond: 4, next: [0,5], pr: 0.012}
  (19,8):
    - {tocond: 3, next: [0,0], pr: 0.922}
    - {tocond: 8, next: [0,5], pr: 0.072}
    - {tocond: 4, next: [0,5], pr: 0.006}
  (25,7):
    - {tocond: 3, next: [0,0], pr: 0.964}
    - {tocond: 4, next: [0,5], pr: 0.036}
  (25,8):
    - {tocond: 3, next: [0,0], pr: 0.964}
    - {tocond: 4, next: [0,5], pr: 0.036}
"""

"3:\n  (5,5):\n    - {tocond: 5, next: [9, 5], pr: 0.2}\n    - {tocond: 6, next: [9, 6], pr: 0.7}\n    - {tocond: 7, next: [9, 7], pr: 0.1}\n  (9,5):\n    - {tocond: 3, next: [0,0], pr: 0.85}\n    - {tocond: 7, next: [14, 7], pr: 0.15}\n  (9,6):\n    - {tocond: 6, next: [14,6], pr: 1.0}\n  (9,7):\n    - {tocond: 7, next: [14, 7], pr: 0.9}\n    - {tocond: 8, next: [14, 8], pr: 0.1}\n  (14,6):\n    - {tocond: 3, next: [0,0], pr: 1.0}\n  (14,7):\n    - {tocond: 3, next: [0, 0], pr: 0.83}\n    - {tocond: 8, next: [19, 7], pr: 0.1}\n    - {tocond: 8, next: [19, 8], pr: 0.07}\n  (14,8):\n    - {tocond: 3, next: [0,0], pr: 0.474}\n    - {tocond: 8, next: [19, 8], pr: 0.514}\n    - {tocond: 4, next: [0,5], pr: 0.012}\n  (19,8):\n    - {tocond: 3, next: [0,0], pr: 0.922}\n    - {tocond: 8, next: [0,5], pr: 0.072}\n    - {tocond: 4, next: [0,5], pr: 0.006}\n  (25,7):\n    - {tocond: 3, next: [0,0], pr: 0.964}\n    - {tocond: 4, next: [0,5], pr: 0.036}\n  (25,8):\n    - {tocond: 3, next: [0,0], p

In [47]:
hand_test = YAML.load(hand_yaml)

Dict{Any,Any} with 1 entry:
  3 => Dict{Any,Any}("(9,6)"=>Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"p…

In [67]:
dense_test = YAML.load_file("../parameters/dec_tree_all_25.yml")

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}([9, 5]=>Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"…
  2 => Dict{Any,Any}([9, 5]=>Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"…
  3 => Dict{Any,Any}([9, 5]=>Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"…
  5 => Dict{Any,Any}([9, 5]=>Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"…
  1 => Dict{Any,Any}([9, 5]=>Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"…

In [62]:
decpoints = Dict{Int,Array{Int, 1}}()
for i in 1:5
    decpoints[i] = unique([k[1] for k in keys(dense_test[i])])
end
decpoints

Dict{Int64,Array{Int64,1}} with 5 entries:
  4 => [9, 25, 14, 5, 19]
  2 => [9, 25, 14, 5, 19]
  3 => [9, 25, 14, 5, 19]
  5 => [9, 25, 14, 5, 19]
  1 => [9, 25, 14, 5, 19]

In [63]:
display_tree(dense_test[1])

[5, 5]
   Dict{Any,Any}("tocond" => 5,"next" => [9, 5],"pr" => 0.4)
   Dict{Any,Any}("tocond" => 6,"next" => [9, 6],"pr" => 0.5)
   Dict{Any,Any}("tocond" => 7,"next" => [9, 7],"pr" => 0.1)
[9, 5]
   Dict{Any,Any}("tocond" => 3,"next" => [0, 0],"pr" => 0.9)
   Dict{Any,Any}("tocond" => 7,"next" => [14, 7],"pr" => 0.1)
[9, 6]
   Dict{Any,Any}("tocond" => 6,"next" => [14, 6],"pr" => 1.0)
[9, 7]
   Dict{Any,Any}("tocond" => 7,"next" => [14, 7],"pr" => 0.95)
   Dict{Any,Any}("tocond" => 8,"next" => [14, 8],"pr" => 0.05)
[14, 6]
   Dict{Any,Any}("tocond" => 3,"next" => [0, 0],"pr" => 1.0)
[14, 7]
   Dict{Any,Any}("tocond" => 3,"next" => [0, 0],"pr" => 0.85)
   Dict{Any,Any}("tocond" => 7,"next" => [25, 7],"pr" => 0.12)
   Dict{Any,Any}("tocond" => 8,"next" => [19, 8],"pr" => 0.03)
[14, 8]
   Dict{Any,Any}("tocond" => 3,"next" => [0, 0],"pr" => 0.692)
   Dict{Any,Any}("tocond" => 8,"next" => [19, 8],"pr" => 0.302)
   Dict{Any,Any}("tocond" => 4,"next" => [0, 5],"pr" => 0.006)
[19, 8]
   Dict

In [64]:
dense_test[4]

Dict{Any,Any} with 10 entries:
  [9, 5]  => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.62), Dict("t…
  [25, 8] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.688), Dict("…
  [9, 7]  => Dict{Any,Any}[Dict("tocond"=>7,"next"=>[14, 7],"pr"=>0.78), Dict("…
  [14, 7] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.8), Dict("to…
  [5, 5]  => Dict{Any,Any}[Dict("tocond"=>5,"next"=>[9, 5],"pr"=>0.15), Dict("t…
  [9, 6]  => Dict{Any,Any}[Dict("tocond"=>6,"next"=>[14, 6],"pr"=>1.0)]
  [14, 8] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.165), Dict("…
  [19, 8] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.81), Dict("t…
  [25, 7] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.76), Dict("t…
  [14, 6] => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>1.0)]

In [65]:
length(dense_test)

5

In [66]:
CovidSim.sanity_test_all(dense_test)

5×4 Array{Float64,2}:
 1.0  1.0  0.999511  0.000488522
 2.0  1.0  0.999332  0.000668336
 3.0  1.0  0.99616   0.0038404
 4.0  1.0  0.980863  0.0191366
 5.0  1.0  0.850242  0.149758

In [10]:
std_file = "../parameters/dec_tree_all_25.yml"

"../parameters/dec_tree_all_25.yml"

In [13]:
treedict, decpoints = setup_dt(std_file)

(Dict{Any,Any}(4 => Dict{Any,Any}([9, 5] => Dict{String,Array{T,1} where T}("probs" => [0.62, 0.38],"branches" => Dict{Any,Any}[Dict("tocond" => 3,"next" => [0, 0],"pr" => 0.62), Dict("tocond" => 7,"next" => [14, 7],"pr" => 0.38)],"outcomes" => [3, 7]),[25, 8] => Dict{String,Array{T,1} where T}("probs" => [0.688, 0.312],"branches" => Dict{Any,Any}[Dict("tocond" => 3,"next" => [0, 0],"pr" => 0.688), Dict("tocond" => 4,"next" => [0, 5],"pr" => 0.312)],"outcomes" => [3, 4]),[9, 7] => Dict{String,Array{T,1} where T}("probs" => [0.78, 0.22],"branches" => Dict{Any,Any}[Dict("tocond" => 7,"next" => [14, 7],"pr" => 0.78), Dict("tocond" => 8,"next" => [14, 8],"pr" => 0.22)],"outcomes" => [7, 8]),[14, 7] => Dict{String,Array{T,1} where T}("probs" => [0.8, 0.1, 0.1],"branches" => Dict{Any,Any}[Dict("tocond" => 3,"next" => [0, 0],"pr" => 0.8), Dict("tocond" => 8,"next" => [25, 7],"pr" => 0.1), Dict("tocond" => 8,"next" => [19, 8],"pr" => 0.1)],"outcomes" => [3, 8, 8]),[5, 5] => Dict{String,Array{T

In [14]:
treedict

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.62, 0.…
  2 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.85, 0.…
  3 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…
  5 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.5, 0.5…
  1 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…

In [16]:
treedict[1][[9,5]]

Dict{String,Array{T,1} where T} with 3 entries:
  "probs"    => [0.9, 0.1]
  "branches" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.9), Dict(…
  "outcomes" => [3, 7]

In [17]:
decpoints

Dict{Int64,Array{Int64,1}} with 5 entries:
  4 => [9, 25, 14, 5, 19]
  2 => [9, 25, 14, 5, 19]
  3 => [9, 25, 14, 5, 19]
  5 => [9, 25, 14, 5, 19]
  1 => [9, 25, 14, 5, 19]